## Introduction 

This lesson will cover: 
- 甚麼是函數調用及其使用案例 
- 如何使用 OpenAI 創建函數調用 
- 如何將函數調用整合到應用程式中 

## Learning Goals 

After completing this lesson you will know how to and understand: 

- 使用函數調用的目的 
- 使用 OpenAI 服務設置函數調用 
- 為您的應用程式使用案例設計有效的函數調用 


## 理解函數調用

在本課程中，我們希望為我們的教育初創公司建立一個功能，讓用戶可以使用聊天機械人來尋找技術課程。我們將推薦符合他們技能水平、當前職位和感興趣技術的課程。

為了完成這個，我們將結合使用：
 - `OpenAI` 為用戶創建聊天體驗
 - `Microsoft Learn Catalog API` 幫助用戶根據用戶的請求尋找課程
 - `Function Calling` 將用戶的查詢發送到函數以進行 API 請求。

首先，讓我們看看為什麼我們首先想使用函數調用：

print("下一次請求中的訊息：")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # 從 GPT 獲取新回應，GPT 可以看到函數回應


print(second_response.choices[0].message)


### 為什麼要使用函數調用

如果你已經完成了本課程中的其他課程，你大概已經了解使用大型語言模型（LLMs）的強大功能。希望你也能看到它們的一些限制。

函數調用是 OpenAI 服務的一項功能，旨在解決以下挑戰：

回應格式不一致：
- 在函數調用之前，大型語言模型的回應是無結構且不一致的。開發者必須編寫複雜的驗證代碼來處理輸出中的每種變化。

與外部數據整合有限：
- 在此功能之前，將應用程式其他部分的數據整合到聊天上下文中是困難的。

通過標準化回應格式並實現與外部數據的無縫整合，函數調用簡化了開發流程，並減少了額外驗證邏輯的需求。

用戶無法獲得像「斯德哥爾摩目前的天氣如何？」這樣的答案。這是因為模型的知識僅限於訓練數據的時間範圍。

讓我們看下面的範例來說明這個問題：

假設我們想建立一個學生數據庫，以便能夠向他們推薦合適的課程。以下是兩個學生的描述，它們在所包含的數據上非常相似。


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

我哋想將呢啲資料發送畀一個大型語言模型（LLM）去解析。呢啲資料日後可以用喺我哋嘅應用程式，去發送畀API或者儲存喺數據庫入面。

我哋嚟創建兩個完全相同嘅提示，指示LLM我哋有興趣嘅資訊係啲乜：


我哋想將呢個發送畀一個大型語言模型（LLM），去解析對我哋產品重要嘅部分。所以我哋可以創建兩個相同嘅提示，去指示呢個LLM：


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


在創建這兩個提示後，我們將使用 `openai.ChatCompletion` 將它們發送給大型語言模型（LLM）。我們將提示存儲在 `messages` 變量中，並將角色分配為 `user`。這是為了模擬用戶向聊天機器人發送消息。


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

現在我們可以將兩個請求同時發送給大型語言模型，並檢查我們收到的回應。


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

即使提示相同且描述相似，我們仍可能獲得不同格式的 `Grades` 屬性。

如果多次執行上述儲存格，格式可能是 `3.7` 或 `3.7 GPA`。

這是因為 LLM 接收以書面提示形式呈現的非結構化數據，並且也返回非結構化數據。我們需要有一個結構化格式，這樣在儲存或使用這些數據時才能知道會得到什麼。

透過使用函數調用，我們可以確保收到結構化的數據。使用函數調用時，LLM 並不會實際調用或執行任何函數。相反，我們為 LLM 創建一個結構，讓它依此結構回應。然後，我們使用這些結構化的回應來知道在應用程式中要執行哪個函數。


![函數調用流程圖](../../../../translated_images/Function-Flow.083875364af4f4bb.mo.png)


然後，我們可以將函數返回的結果發送回大型語言模型（LLM）。LLM 將使用自然語言回應，以回答用戶的查詢。


### 使用函數調用的用例

**調用外部工具**  
聊天機器人在為用戶提供問題答案方面表現出色。通過使用函數調用，聊天機器人可以利用用戶的訊息來完成某些任務。例如，學生可以請聊天機器人「發送電郵給我的導師，說我需要更多這科的協助」。這可以調用函數 `send_email(to: string, body: string)`

**建立 API 或資料庫查詢**  
用戶可以使用自然語言查找資訊，並將其轉換為格式化的查詢或 API 請求。舉例來說，老師可以請求「誰完成了最後的作業」，這可以調用名為 `get_completed(student_name: string, assignment: int, current_status: string)` 的函數

**建立結構化資料**  
用戶可以將一段文字或 CSV 檔案，利用大型語言模型提取重要資訊。例如，學生可以將一篇關於和平協議的維基百科文章轉換成 AI 快速記憶卡。這可以通過調用函數 `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` 來完成


## 2. 建立你的第一個函數調用

建立函數調用的過程包括三個主要步驟：
1. 使用你的函數列表和用戶訊息調用 Chat Completions API
2. 閱讀模型的回應以執行動作，即執行函數或 API 調用
3. 使用你的函數回應，再次調用 Chat Completions API，利用該資訊來建立對用戶的回應。


![函數調用流程](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.mo.png)


### 函數呼叫的元素

#### 使用者輸入

第一步是建立使用者訊息。這可以透過取得文字輸入的值動態指派，或者你也可以在這裡指派一個值。如果這是你第一次使用 Chat Completions API，我們需要定義訊息的 `role` 和 `content`。

`role` 可以是 `system`（建立規則）、`assistant`（模型）或 `user`（最終使用者）。對於函數呼叫，我們會將其指派為 `user` 並給出一個範例問題。


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### 建立函數。

接下來我們會定義一個函數及該函數的參數。我們這裡只會使用一個名為 `search_courses` 的函數，但你可以建立多個函數。

**重要**：函數會包含在系統訊息中傳送給大型語言模型，並會計入你可用的令牌數量。


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**定義** 

函數定義結構有多個層級，每個層級都有其自身的屬性。以下是巢狀結構的解析：

**頂層函數屬性：**

`name` - 我們希望被呼叫的函數名稱。

`description` - 函數如何運作的描述。在這裡，具體且清晰非常重要。

`parameters` - 您希望模型在回應中產生的值和格式清單。

**參數物件屬性：**

`type` - 參數物件的資料類型（通常為 "object"）

`properties` - 模型將用於回應的具體值清單。

**個別參數屬性：**

`name` - 由屬性鍵隱含定義（例如 "role"、"product"、"level"）

`type` - 此特定參數的資料類型（例如 "string"、"number"、"boolean"）

`description` - 該特定參數的描述。

**可選屬性：**

`required` - 列出完成函數呼叫所需參數的陣列。


### 呼叫函數  
定義函數後，我們現在需要在呼叫 Chat Completion API 時包含它。我們透過在請求中加入 `functions` 來做到這點。在這個例子中是 `functions=functions`。

也有一個選項可以將 `function_call` 設定為 `auto`。這表示我們會讓大型語言模型根據使用者訊息決定應該呼叫哪個函數，而不是由我們自己指定。


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

現在讓我們來看看回應的格式：

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

你可以看到函數的名稱被呼叫了，並且從使用者訊息中，LLM 能夠找到符合函數參數的資料。


## 3.將函數調用整合到應用程式中。

在我們測試了來自大型語言模型（LLM）的格式化回應後，現在可以將其整合到應用程式中。

### 管理流程

要將此整合到我們的應用程式中，讓我們採取以下步驟：

首先，讓我們呼叫 OpenAI 服務並將訊息存儲在名為 `response_message` 的變數中。


In [ ]:
response_message = response.choices[0].message

現在我們將定義一個函數，該函數將調用 Microsoft Learn API 以獲取課程列表：


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



作為最佳實踐，我們接著會查看模型是否想要呼叫一個函數。之後，我們會建立一個可用的函數並將其與正在被呼叫的函數相匹配。  
然後，我們會取得該函數的參數並將它們對應到來自 LLM 的參數。

最後，我們會附加函數呼叫訊息以及由 `search_courses` 訊息返回的值。這樣就能提供 LLM 所需的所有資訊，  
以自然語言回應使用者。


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



現在我們將把更新的訊息發送給大型語言模型，以便我們能收到自然語言的回應，而非API的JSON格式回應。


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code Challenge 

做得好！為了繼續學習 OpenAI Function Calling，你可以建立：https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - 函數的更多參數，可能幫助學習者找到更多課程。你可以在這裡找到可用的 API 參數： 
 - 建立另一個函數調用，從學習者那裡獲取更多資訊，例如他們的母語 
 - 當函數調用和/或 API 調用未返回任何合適的課程時，建立錯誤處理


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：
本文件係使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我哋致力於確保準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件之母語版本應視為權威來源。對於重要資訊，建議採用專業人工翻譯。因使用本翻譯而引致之任何誤解或誤釋，本公司概不負責。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
